In [2]:
import pandas as pd
import numpy as np


In [3]:
data=pd.read_csv(r'C:\Users\Dell\Downloads\final_dataset.csv').dropna()  # Load and clean
data

,text,emotion
0,i feel rather funny ending with so many dupes ...,fun
1,i feel surprised by the result,surprise
2,i am officially feeling festive,neutral
3,i suddenly found myself standing before this w...,surprise
4,i look at the meager pile of food i purchased ...,enthusiasm
...,...,...
106350,i used to feel strongly about how much i hated...,hate
106351,i feel like i just got a spirit booster this r...,surprise
106352,i could come up with is that i was really feel...,anger
106353,i find it really it helps to have an outfit of...,relief


#tokenzier for x and y train


In [5]:
x_tri=data['text']
y_tri=data['emotion'].unique()
y_tri=list(y_tri)
y_tri


['fun',
 'surprise',
 'neutral',
 'enthusiasm',
 'happiness',
 'hate',
 'sadness',
 'empty',
 'love',
 'relief',
 'anger']

In [6]:
import nltk
nltk.download('all')


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_

True

In [7]:
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,WordNetLemmatizer
stemm=PorterStemmer()

In [8]:
stop_words=set(stopwords.words('english'))    

In [9]:
data['stemmed_tokens'] = data['text'].astype(str).str.lower().apply(
    lambda x: [stemm.stem(word) for word in word_tokenize(x)]
)

In [10]:
data['stemmed_tokens'] 
data['processed_text'] = data['stemmed_tokens'].apply(lambda x: ' '.join([word for word in x if word not in stop_words]))
data['processed_text']

0         feel rather funni end mani dupe alway prefer o...
1                                       feel surpris result
2                                        offici feel festiv
3         suddenli found stand befor thi woman dress lik...
4         look meager pile food purchas week feel pretti...
                                ...                        
106350    use feel strongli much hate curli feel strongl...
106351    feel like got spirit booster thi refer amaz ne...
106352    could come wa realli feel victim made absolut ...
106353    find realli help outfit favourit basic piec fe...
106354                     help feel surpris hi sudden call
Name: processed_text, Length: 106355, dtype: object

In [11]:
# Correct the train_test_split assignment in the cell with the split:
import re
x = data['processed_text']
name = x.apply(lambda x: re.sub(r'[^\w\s]', '', x))
y = data['emotion']
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# vectorization

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(  
    stop_words='english',
    ngram_range=(1, 2),
    min_df=3,
    max_df=0.9,
    sublinear_tf=True,
    max_features=25000)
x_train_vectorized = vectorizer.fit_transform(x_train)
x_train_vectorized.min()
x_test_vectorized = vectorizer.transform(x_test)



In [13]:
from sklearn.svm import LinearSVC
model = LinearSVC(C=2.5, max_iter=300)
model.fit(x_train_vectorized, y_train)
y_pred=model.predict(x_test_vectorized)
y_pred

array(['neutral', 'fun', 'fun', ..., 'anger', 'empty', 'hate'],
      dtype=object)

In [14]:
from sklearn.metrics import accuracy_score
y_pred_test = model.predict(x_test_vectorized)
accuracy = accuracy_score(y_test, y_pred_test)
print(f"Model accuracy on test set: {accuracy:.4f}")

Model accuracy on test set: 0.8437


In [15]:
y_pred_train = model.predict(x_train_vectorized)
train_accuracy = accuracy_score(y_train, y_pred_train)
print(f"Model accuracy on training set: {train_accuracy:.4f}")

Model accuracy on training set: 0.9679


In [16]:
import joblib

model_artifact = {
    "vectorizer": vectorizer,
    "model": model
}
joblib.dump(model_artifact, "tfidf_svm_model.pkl")

['tfidf_svm_model.pkl']